In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################

from flask import Flask, render_template, url_for, session, flash, request, redirect
from datetime import datetime
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
import sqlite3


app = Flask(__name__)






app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db' 
app.config['SECRET_KEY'] = 'thisisasecretkey'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    
class classes(db.Model):
    __tablename__ = 'classes'
    name = db.Column(db.String(25), primary_key=True)
    img = db.Column(db.Text)
    url = db.Column(db.Text)

#class hiphop(db.Model):
   # __tablename__ = 'hiphop'
   # name = db.Column(db.String(25), primary_key=True)
   # ig = db.Column(db.Text)
   # regularclass = db.Column(db.Text) 
   # imgurl=db.Column(db.Text)
    
        
    def __repr__(self):
        return '<User %r>' %self.id
    
class Contact (db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    name=db.Column(db.String(50))   
    email = db.Column(db.String(50))    
    phonenumber=db.Column(db.String(50))
    message=db.Column(db.String(1000))    

db.create_all()
    
class RegisterForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=1, max=15)], render_kw={"placeholder": "Username"})
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=1, max=15)], render_kw={"placeholder": "Password"})
    submit = SubmitField("Register")
    
    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")

            
class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=1, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=1, max=15)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")
    

class ContactForm(FlaskForm):
    name = StringField("Name", validators=[InputRequired(), Length(min=1, max=20)], render_kw={"placeholder": "Name"})   
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    phonenumber = StringField("Phonenumber", validators=[InputRequired(),Length(min=1, max=20)], render_kw={"placeholder": "Phone Number"})
    message=StringField("Message", validators=[InputRequired(), Length(min=1, max=1000)], render_kw={"placeholder": "Message"})
    submit = SubmitField("Submit")
################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

################################################################################################################


@app.route("/")
def index():
    return render_template('index.html')

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/workshop")
def workshop():
    return render_template('workshop.html')

@app.route("/competition")
def competition():
    return render_template('competition.html')

@app.route("/shop")
def shop():
    return render_template('shop.html')


@app.route("/workshop_member")
def workshop_member():
    return render_template('workshop_member.html')

@app.route("/competition_member")
def competition_member():
    return render_template('competition_member.html')

@app.route("/shop_member")
def shop_member():
    return render_template('shop_member.html')

@app.route("/hiphop")
def hiphop():
    return render_template('hiphop.html')

@app.route("/jazz")
def jazz():
    return render_template('jazz.html')

@app.route("/popping")
def popping():
    return render_template('popping.html')

@app.route("/locking")
def locking():
    return render_template('locking.html')

@app.route("/breaking")
def breaking():
    return render_template('breaking.html')

@app.route("/waacking")
def waacking():
    return render_template('waacking.html')

@app.route("/sent")
def sent():
    return render_template('sent.html')

#@app.route("/classes")
#def classes():    
    #return render_template('classes.html')

#@app.route("/dashboard")
#def dashboard():
    #return render_template('dashboard.html')
 
@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user) 
                return redirect(url_for("competition_member"))   
            
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/logout', methods=["GET","POST"])
@login_required
def logout():
    logout_user()
    return redirect(url_for('index'))


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data,password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html',form=form)
    
@app.route("/classes")
def classes():

    sqlalchemyObj = db.engine.execute('select * from classes ORDER BY Name')
    classes = []
    for i in sqlalchemyObj:
        classes.append(i)
    # print(venues)
    dataset = []
    dict={}
    for i in classes:
        dict['name'] = i[0]
        dict['img'] = i[1]
        dict['url'] = i[2]
        
        #dict['Img_url'] = i[4]
        #dict['Desc'] = i[5]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    #print(dataset)

    return render_template('classes.html', title='classes', dataset=dataset)


    
@app.route('/contact', methods=['GET','POST'])
def contact():
    form = ContactForm()
    if form.validate_on_submit():       
        new_message = Contact(name=form.name.data,email=form.email.data,phonenumber=form.phonenumber.data,message=form.message.data)
        db.session.add(new_message)
        db.session.commit()
        return redirect(url_for('sent'))
       
    return render_template('contact.html',form=form)
    
     #   dataset.append(dict.copy()) #markers.append(fld.copy())
  #  print(dataset)

   # return render_template('classes.html', title='Gallery', dataset=dataset)
       
    
# ERROR HANDLERS

################################################################################################################
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


################################################################################################################

# APPLICATION TEST RUN AT PORT 9003

################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 2016)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:2016/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 16:21:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:21:52] "GET /shop_member HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:21:52] "GET /competition_member HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:22:37] "GET /competition HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:22:42] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:23:11] "GET /workshop_member HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:23:37] "GET /workshop HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:23:49] "GET /shop_member HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:24:49] "GET /classes HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:24:49] "GET /static/assets/img/classes.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:24:50] "GET /hiphop HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:24:50] "GET /static/assets/img/hiphop/IMG_2420.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 16:24:50] "GET /static/assets/img/h